# Import files

In [1]:
import data_utils
import training_utils
import RandomAgent
import EpsilonGreedy
import UCB
import Thompson
import Exp3
import FairUCB
import movielens_utils

from data_utils import *
from training_utils import *
from RandomAgent import *
from EpsilonGreedy import *
from UCB import *
from Thompson import *
from Exp3 import *
from FairUCB import *
from movielens_utils import *

In [2]:
np.set_printoptions(threshold = np.inf)
plt.rcParams['figure.figsize'] = [15, 14]

# Dataset Utilities (Simulated Environment)

## An example dataset

Now let us illustrate what we did previously with a simple example and check that it's all working.
In the following, we generate 100 rows of data for a user with a clear preference: they like 100% of the content from group 1 and nothing from any other group (i.e, $p_1 = 1$ and $p_2 = 0$).

In [ ]:
example = new_user(user_id = 0, n = 1000, n_arms = 5, print_stats = True)

As expected, this user likes all items from group 10 and none from other groups.

# Training Utilities

Now that we have a dataset, we need to construct a simulation environment to use for training the bandit. In traditional machine learning, we build training/validation/test sets, where a model is trained and tuned on the training/validation sets and evaluated on the test set.

However, bandit algorithms are different in that they learn over time (*online*). At each time step, the bandit observes data from the past, updates its decision rule (called a *policy*), takes action by serving predictions based on this updated policy, and observes a reward value for this new action. The time component implies that the training data observed by the bandit is constantly changing, and that the score used to evaluate it is also changing over time based on small pieces of feedback from the most recent time step, rather than based on feedback from a large test set as is the case with traditional approaches.

The bandit’s recommendations will be different from those from our data generation model above. This implies several issues:
* First, the data is probably biased. An online learner requires a feedback loop where it presents an action, observes a user’s response, and then updates its policy accordingly. A historic dataset is going to be biased by the mechanism that generated it. The algorithm assumes that it generated the recommendation, but in reality, everything in the dataset was generated by a completely different model. A solution to this is to randomize the data generation process to create a dataset that’s independent and identically distributed, which we did at the beginning of this notebook.
* Second, the learning algorithm often generates recommendations that are different from the recommendations seen by users in the historic dataset. We can't supply a reward value for these recommendations because we don’t know what the user’s response would have been to something they never saw. We can only know how a user responded to what was supplied to them (i.e, the rows in the generated dataset). The solution to this is a method called **replay** [(Li et al., 2010)](https://arxiv.org/abs/1003.5956). Replay evaluation essentially takes the historic event stream and the algorithm’s recommendations at each time step, and throws out all samples except those where the model’s recommendation is the same as the one the user saw in the generated dataset. This proves to be an unbiased method for offline evaluation of an online learing algorithm.

# Bandit Algorithms

In this section, we implement 4 bandit algorithms:
* **$\epsilon$-greedy:** this algorithm follows a greedy policy, selecting the best-performing action (i.e, the one yielding the highest reward) at each time step. However, $\epsilon$% of the time, it goes off-policy and chooses an action at random. The value of $\epsilon$ determines how much the bandit should explore new actions as opposed to exploiting the ones that have performed the best historically up to now.
* **Upper Confidence Bound (UBC):** this algorithm constructs a confidence interval of what each action’s true reward might be, factoring in the uncertainty caused by variance in the data and the fact that we’re only able to observe a limited sample of trials for any given action. The algorithm then optimistically assumes that each action will perform as well as its upper confidence bound (UCB), selecting the one with the highest UCB.
* **Thompson Sampling:** this algorithm draws from the posterior predictive distributions of each choice using a random uniform variable. This allows a non-optimal distribution to be sampled with varying frequency — as the posterior distribution becomes more certain, the probability of the choice being made decreases dynamically; thus Thompson sampling adaptively balances the desire for more information with making the currently optimal choice.
* **Exp3:**

# Experiments

### Global parameters

In [ ]:
# Size of each user dataset:
m = 1000

# Number of training rounds for each agent:
rounds = 50

# Number of arms:
arms = 5

# Optimal agent parameters after tuning:
param_gamma      = 0.1 # Thompson Sampling
param_epsilon    = 0.1 # Epsilon-Greedy
param_multiplier = 1.0 # UCB

# 1. Baseline: Stochastic Bandits

# A. The Natural Setting (without polarization correction)

## First User: Random Preferences ($p_i = 0.5 \ \forall \ i$)

### Generate user data

In [ ]:
user = new_user(1, n = m, n_arms = arms, preferences = [0.5, 0.5, 0.5, 0.5, 0.5], print_stats = True)

### Agents initialization and training

In [ ]:
# Initialize agents
rdm  = RandomBandit()
eps  = EpsilonGreedy(epsilon = param_epsilon)
ucb  = UCB(bonus_multiplier = param_multiplier)
th   = Thompson()
exp3 = Exp3(gamma = param_gamma)

# Train agents
history_rdm  = train(user,  rdm, steps = rounds)
history_eps  = train(user,  eps, steps = rounds)
history_ucb  = train(user,  ucb, steps = rounds)
history_th   = train(user,   th, steps = rounds)
history_exp3 = train(user, exp3, steps = rounds)

### Result plots

### Random agent

In [ ]:
score(history_rdm, n_arms = arms)

### $\epsilon$-Greedy

In [ ]:
score(history_eps, n_arms = arms)

### UCB

In [ ]:
score(history_ucb, n_arms = arms)

### Thompson Sampling

In [ ]:
score(history_th, n_arms = arms)

### Exp3

In [ ]:
score(history_exp3, n_arms = arms)

## Second User: Likes group #1, nothing else ($p_5 = 1$, $p_i = 0 \ \forall \ i < 5$)

### Generate user data

In [ ]:
user2 = new_user(2, n = m, n_arms = arms, preferences = [1, 0, 0, 0, 0], print_stats = True)

### Agents initialization and training

In [ ]:
# Initialize agents
eps2   = EpsilonGreedy(epsilon = param_epsilon)
ucb2   = UCB(bonus_multiplier = param_multiplier)
th2    = Thompson()
exp3_2 = Exp3(gamma = param_gamma)

# Train agents
history_rdm2   = train(user2,    rdm, steps = rounds)
history_eps2   = train(user2,   eps2, steps = rounds)
history_ucb2   = train(user2,   ucb2, steps = rounds)
history_th2    = train(user2,    th2, steps = rounds)
history_exp3_2 = train(user2, exp3_2, steps = rounds)

### Random Agent

In [ ]:
score(history_rdm2, n_arms = arms)

### $\epsilon$-Greedy

In [ ]:
score(history_eps2, n_arms = arms)

### UCB

In [ ]:
score(history_ucb2, n_arms = arms)

### Thompson Sampling

In [ ]:
score(history_th2, n_arms = arms)

### Exp3

In [ ]:
score(history_exp3_2, n_arms = arms)

## Third User: prefers groups #1 and #2 ($p_1 = p_2 = 0.9$, $p_i = 0.5 \ \forall \ i > 2$)

### Generate user data

In [ ]:
user3 = new_user(3, n = m, n_arms = arms, preferences = [0.9, 0.9, 0.5, 0.5, 0.5], print_stats = True)

### Agents initialization and training

In [ ]:
# Initialize agents
eps3   = EpsilonGreedy(epsilon = param_epsilon)
ucb3   = UCB(bonus_multiplier = param_multiplier)
th3    = Thompson()
exp3_3 = Exp3(gamma = param_gamma)

# Train agents
history_rdm3   = train(user3,    rdm, steps = rounds)
history_eps3   = train(user3,   eps3, steps = rounds)
history_ucb3   = train(user3,   ucb3, steps = rounds)
history_th3    = train(user3,    th3, steps = rounds)
history_exp3_3 = train(user3, exp3_3, steps = rounds)

### Random Agent

In [ ]:
score(history_rdm3, n_arms = arms)

### $\epsilon$-Greedy

In [ ]:
score(history_eps3, n_arms = arms)

### UCB

In [ ]:
score(history_ucb3, n_arms = arms)

### Thompson Sampling

In [ ]:
score(history_th3, n_arms = arms)

### Exp3

In [ ]:
score(history_exp3_3, n_arms = arms, rolling = 50)

# B. The Depolarized Setting

## First User: Random Preferences ($p_i = 0.5 \ \forall \ i$)

### Agents initialization and training

In [ ]:
# Initialize agents
eps  = EpsilonGreedy(epsilon = param_epsilon, depolarize = True, upper = 0.6)
ucb  = UCB(bonus_multiplier = param_multiplier, depolarize = True, upper = 0.6)
th   = Thompson(depolarize = True, upper = 0.6)
exp3 = Exp3(gamma = param_gamma, depolarize = True, upper = 0.6)

# Train agents
history_eps  = train(user,  eps, steps = rounds)
history_ucb  = train(user,  ucb, steps = rounds)
history_th   = train(user,   th, steps = rounds)
history_exp3 = train(user, exp3, steps = rounds)

### $\epsilon$-Greedy

In [ ]:
score(history_eps, n_arms = arms)

### UCB

In [ ]:
score(history_ucb, n_arms = arms)

### Thompson Sampling

In [ ]:
score(history_th, n_arms = arms)

### Exp3

In [ ]:
score(history_exp3, n_arms = arms)

## Second User: Likes group #1, nothing else ($p_5 = 1$, $p_i = 0 \ \forall \ i < 5$)

### Agents initialization and training

In [ ]:
# Initialize agents
eps2   = EpsilonGreedy(epsilon = param_epsilon, depolarize = True, upper = 0.6)
ucb2   = UCB(bonus_multiplier = param_multiplier, depolarize = True, upper = 0.6)
th2    = Thompson(depolarize = True, upper = 0.6)
exp3_2 = Exp3(gamma = param_gamma, depolarize = True, upper = 0.6)

# Train agents
history_eps2   = train(user2,   eps2, steps = rounds)
history_ucb2   = train(user2,   ucb2, steps = rounds)
history_th2    = train(user2,    th2, steps = rounds)
history_exp3_2 = train(user2, exp3_2, steps = rounds)

### $\epsilon$-Greedy

In [ ]:
score(history_eps2, n_arms = arms)

### UCB

In [ ]:
score(history_ucb2, n_arms = arms)

### Thompson Sampling

In [ ]:
score(history_th2, n_arms = arms)

### Exp3

In [ ]:
score(history_exp3_2, n_arms = arms)

## Third User: prefers groups #1 and #2 ($p_1 = p_2 = 0.9$, $p_i = 0.5 \ \forall \ i > 2$)

### Agents initialization and training

In [ ]:
# Initialize agents
eps3 = EpsilonGreedy(epsilon = param_epsilon, depolarize = True, upper = 0.6)
ucb3 = UCB(bonus_multiplier = param_multiplier, depolarize = True, upper = 0.6)
th3 = Thompson(depolarize = True, upper = 0.6)
exp3_3 = Exp3(gamma = param_gamma, depolarize = True, upper = 0.6)

# Train agents
history_eps3   = train(user3,   eps3, steps = rounds)
history_ucb3   = train(user3,   ucb3, steps = rounds)
history_th3    = train(user3,    th3, steps = rounds)
history_exp3_3 = train(user3, exp3_3, steps = rounds)

### $\epsilon$-Greedy

In [ ]:
score(history_eps3, n_arms = arms)

### UCB

In [ ]:
score(history_ucb3, n_arms = arms)

### Thompson Sampling

In [ ]:
score(history_th3, n_arms = arms)

### Exp3

In [ ]:
score(history_exp3_3, n_arms = arms)

# 2. Adding Constraints

# A UCB variant with fair group representations

In [ ]:
# Initialize agent
fairUCB = FairUCB(lower = 0.10)

# Train agent
fairHistory = train(user2, fairUCB, steps = rounds)

# Plot results
score(fairHistory, n_arms = arms)

# Combining lower and upper fairness constraints

In [ ]:
# Initialize agent
fairUCB2 = FairUCB(lower = 0.1, upper = 0.5)

# Train agent
fairHistory = train(user2, fairUCB2, steps = rounds)

# Plot results
score(fairHistory, n_arms = arms)

# 3. Fair-MAB

# MovieLens Dataset

In [3]:
movielens = getdata()